## Prediction of Credibility of Disaster Tweets

In [1]:
import pandas as pd
import re
from sklearn import linear_model,feature_extraction,model_selection
from sklearn.feature_extraction import text

In [2]:
train_df=pd.read_csv("C:/Users/madir/Downloads/train.csv")
test_df=pd.read_csv("C:/Users/madir/Downloads/test.csv")

In [3]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
train_df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [5]:
train_df["text"]

0       Our Deeds are the Reason of this #earthquake M...
1                  Forest fire near La Ronge Sask. Canada
2       All residents asked to 'shelter in place' are ...
3       13,000 people receive #wildfires evacuation or...
4       Just got sent this photo from Ruby #Alaska as ...
5       #RockyFire Update => California Hwy. 20 closed...
6       #flood #disaster Heavy rain causes flash flood...
7       I'm on top of the hill and I can see a fire in...
8       There's an emergency evacuation happening now ...
9       I'm afraid that the tornado is coming to our a...
10            Three people died from the heat wave so far
11      Haha South Tampa is getting flooded hah- WAIT ...
12      #raining #flooding #Florida #TampaBay #Tampa 1...
13                #Flood in Bago Myanmar #We arrived Bago
14      Damage to school bus on 80 in multi car crash ...
15                                         What's up man?
16                                          I love fruits
17            

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
id          7613 non-null int64
keyword     7552 non-null object
location    5080 non-null object
text        7613 non-null object
target      7613 non-null int64
dtypes: int64(2), object(3)
memory usage: 208.2+ KB


In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
id          3263 non-null int64
keyword     3237 non-null object
location    2158 non-null object
text        3263 non-null object
dtypes: int64(1), object(3)
memory usage: 63.8+ KB


In [8]:
test_df["text"]

0                      Just happened a terrible car crash
1       Heard about #earthquake is different cities, s...
2       there is a forest fire at spot pond, geese are...
3                Apocalypse lighting. #Spokane #wildfires
4           Typhoon Soudelor kills 28 in China and Taiwan
5                      We're shaking...It's an earthquake
6       They'd probably still show more life than Arse...
7                                       Hey! How are you?
8                                        What a nice hat?
9                                               Fuck off!
10                                  No I don't like cold!
11                             NOOOOOOOOO! Don't do that!
12                                 No don't tell me that!
13                                              What if?!
14                                               Awesome!
15      Birmingham Wholesale Market is ablaze BBC News...
16      @sunkxssedharry will you wear shorts for race ...
17      #Previ

In [26]:
train_df=train_df.drop_duplicates(["text","target"])

In [27]:
def replace_terms(data):
    for word in data.split():
        word=word.lower()
        value1=re.match(".*@.*",word)
        if value1:
            data = re.sub("@","@ ",data)
        value2=re.match(".*:.*",word)
        if value2:
            data = re.sub(":"," : ",data)
        value3=re.match(".*#.*",word)
        if value3:
            data= re.sub("#","# ",data)
        value4=re.match(".*/.*",word)
        if value4:
            data= re.sub("/"," / ",data)
        value5=re.match(".*_.*",word)
        if value5:
            data= re.sub("_"," _ ",data)
        value6=re.match(".*[0-9]+[a-zA-Z][a-zA-Z]",word)
        if value6:
            index=0
            for i in range(len(word)):
                if word[i] not in ["0","1","2","3","4","5","6","7","8","9"]:
                    index=i
                    break
            word1=word[:index]
            data=re.sub(re.escape(word)," "+word1+" ",data)

    return data

In [28]:
Td_idf_transformer_1=text.TfidfVectorizer(preprocessor=replace_terms)

In [29]:
train_x=Td_idf_transformer_1.fit_transform(train_df["text"])
test_x=Td_idf_transformer_1.transform(test_df["text"])
train_y=train_df["target"]

In [30]:
Model=linear_model.RidgeClassifier()
Model.fit(train_x,train_y)

RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=None,
                solver='auto', tol=0.001)

In [31]:
scores = model_selection.cross_val_score(Model, train_x, train_y, cv=4)
scores

array([0.7267411 , 0.72195641, 0.72659574, 0.7631719 ])

In [32]:
Model.predict(test_x)

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)